This code block is required to be run to set up R kernel on Colab

In [0]:
!apt-get install libssl-dev > /dev/null
!wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x ./Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -p /conda  > /dev/null 2>&1 
!/conda/bin/conda install -c r r-rstan r-irkernel gxx_linux-64 -y -q > /dev/null 2>&1
!/conda/bin/R -e "IRkernel::installspec(name = 'python2', displayname = 'R', user = FALSE)"  > /dev/null 2>&1
!mkdir /root/.R/
!echo "CXX14FLAGS=-O3 -mtune=native -march=native -Wno-ignored-attributes -Wno-deprecated-declarations" > /root/.R/Makevars
import os
os._exit(00)


--2019-06-03 06:57:05--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70348401 (67M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  67.09M   167MB/s    in 0.4s    

2019-06-03 06:57:06 (167 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [70348401/70348401]



In [0]:
Sys.setenv(PATH= paste("/conda/bin" , Sys.getenv("PATH"), sep = ":" ) )


In [2]:
install.packages("readr") #For reading csv files

also installing the dependencies ‘pkgconfig’, ‘hms’, ‘clipr’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [0]:
Sys.setenv(USE_CXX14 = 0)
library(readr)
df <- read_csv("Audionumeric.csv")

Parsed with column specification:
cols(
  .default = col_double(),
  label = col_character()
)
See spec(...) for full column specifications.


In [0]:
head(df)

0,1,2,3,4,5,6,7,8,9,⋯,142,143,144,145,146,147,148,149,150,label
7,-20,3,-5,13,3,6,1,0,-2,⋯,0,2,5,6,-1,3,0,7,3,bird
5,-1,-2,0,0,4,2,8,-16,3,⋯,-2,-1,0,-3,2,-1,0,-2,-1,bird
-52,35,52,-58,-29,-16,-39,18,95,10,⋯,0,0,1,-1,2,1,-4,1,4,bird
-43,56,31,12,9,13,-12,-4,0,14,⋯,0,21,11,1,-32,0,-7,-23,5,bird
0,0,2,-1,-2,-1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,bird
7,-98,-62,-11,-100,-1,28,-24,35,7,⋯,0,0,0,0,0,0,-1,0,0,bird


In [0]:
df$label <- as.numeric(factor(df$label))  #Encoding the label column

I will try to use XGBoost package as it is much efficient compared to other Tree Based Algorithms.
There is another Package which is very fast and reliable comapared to XGBoost. It is CatBoost. CatBoost works wonderously well with Categorical Features. It can outperform XGBoost by over 50 times in the presence of Categorical Features. But the Audionumeric dataset has only numeric features so CatBoost will perform similar to XGBoost (if not worse).


So , I will be using XGBoost here.

In [3]:
install.packages("xgboost")

also installing the dependency ‘data.table’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘backports’, ‘zeallot’, ‘vctrs’, ‘numDeriv’, ‘SQUAREM’, ‘fansi’, ‘pillar’, ‘KernSmooth’, ‘lava’, ‘assertthat’, ‘glue’, ‘R6’, ‘Rcpp’, ‘tibble’, ‘BH’, ‘plogr’, ‘rpart’, ‘survival’, ‘nnet’, ‘class’, ‘prodlim’, ‘codetools’, ‘iterators’, ‘dplyr’, ‘generics’, ‘gower’, ‘ipred’, ‘lubridate’, ‘purrr’, ‘RcppRoll’, ‘rlang’, ‘tidyr’, ‘tidyselect’, ‘timeDate’, ‘foreach’, ‘ModelMetrics’, ‘recipes’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [0]:
library(xgboost)

Before training, the datatset needs to be split into train and test(or val) sets so as to keep a check on overfitting.
~ XGBoost is prone to overfitting if proper hyperparameter tuning and regularization(for gblinear booster) is are not applied.

In [0]:
tr_ind <- sample(1:nrow(df), 0.8 * nrow(df))
val_ind <- setdiff(1:nrow(df), tr_ind)


X_train <- df[tr_ind, 1:151]
y_train <- df[tr_ind, 'label'] -1

X_test <- df[val_ind, 1:151]
y_test <- df[val_ind, "label"] -1

In [0]:
summary(y_train)
summary(y_test)

     label      
 Min.   :0.000  
 1st Qu.:1.000  
 Median :1.000  
 Mean   :1.311  
 3rd Qu.:2.000  
 Max.   :2.000  

     label      
 Min.   :0.000  
 1st Qu.:1.000  
 Median :1.000  
 Mean   :1.283  
 3rd Qu.:2.000  
 Max.   :2.000  

The above summaries reflect that the train and test datasets are stratified.

XGBoost can accept data  in the following forms:

-- R dataframe 

-- R data.table

-- DMatirx( specifically for XGBoost).

Out of these, DMatrix is the most efficient to work using XGBoost.

In [0]:
dtrain <- xgb.DMatrix(data=as.matrix(X_train) ,  label =as.matrix(y_train))
dval <- xgb.DMatrix(data = as.matrix(X_test), label = as.matrix(y_test))
watchlist <- list(train = dtrain , val = dval)

In [0]:
params <- list(eval_metric ='merror' ,booster = 'gbtree' , objective = 'multi:softmax' , num_class=3) 

Cross-Validation helps to tune hyperparameters to some extent.

In [36]:
xgb_mod <- xgb.cv(params = params,data = dtrain,nrounds = 500, nfold=100, showsd=T, stratified=T, print_every_n=20, early.stop.round=40, maximize=F)

Warning message:
“'early.stop.round' is deprecated.
Use 'early_stopping_rounds' instead.
See help("Deprecated") and help("xgboost-deprecated").”

[1]	train-merror:0.040125+0.012724	test-merror:0.244321+0.426819 
Multiple eval metrics are present. Will use test_merror for early stopping.
Will train until test_merror hasn't improved in 40 rounds.

[21]	train-merror:0.000000+0.000000	test-merror:0.242222+0.426406 
[41]	train-merror:0.000000+0.000000	test-merror:0.251975+0.432318 
Stopping. Best iteration:
[10]	train-merror:0.000000+0.000000	test-merror:0.212099+0.406761



In [0]:
n <- xgb_mod$evaluation_log

In [40]:
summary(n)

      iter       train_merror_mean  train_merror_std    test_merror_mean
 Min.   : 1.00   Min.   :0.000000   Min.   :0.0000000   Min.   :0.2121  
 1st Qu.:13.25   1st Qu.:0.000000   1st Qu.:0.0000000   1st Qu.:0.2321  
 Median :25.50   Median :0.000000   Median :0.0000000   Median :0.2420  
 Mean   :25.50   Mean   :0.001892   Mean   :0.0006979   Mean   :0.2386  
 3rd Qu.:37.75   3rd Qu.:0.000000   3rd Qu.:0.0000000   3rd Qu.:0.2501  
 Max.   :50.00   Max.   :0.040125   Max.   :0.0127239   Max.   :0.2526  
 test_merror_std 
 Min.   :0.4068  
 1st Qu.:0.4202  
 Median :0.4264  
 Mean   :0.4240  
 3rd Qu.:0.4309  
 Max.   :0.4323  

Now, training the models with the obtained parameters.

In [53]:
model <- xgb.train(data=dtrain, watchlist = watchlist,nrounds = which.min(n$test_merror_mean), nfolds  =50, objective= 'multi:softmax', eta=0.08, max_depth=15, verbose=1, num_class=3)

[1]	train-merror:0.011111	val-merror:0.239130 
[2]	train-merror:0.011111	val-merror:0.217391 
[3]	train-merror:0.005556	val-merror:0.217391 
[4]	train-merror:0.000000	val-merror:0.239130 
[5]	train-merror:0.000000	val-merror:0.217391 
[6]	train-merror:0.000000	val-merror:0.173913 
[7]	train-merror:0.000000	val-merror:0.195652 
[8]	train-merror:0.000000	val-merror:0.217391 
[9]	train-merror:0.000000	val-merror:0.217391 
[10]	train-merror:0.000000	val-merror:0.217391 


In [0]:
y_pred <- predict(model , dval)  #Predictions

In [69]:
xgb.save(model, "AudClass.model")

[1] TRUE